# Ethereum and ERC20 derivation and transaction manual

In [1]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy

from crypto_eth import HDPrivateKey, HDKey

from binascii import hexlify

import pandas as pd

from IPython.display import display
from ipywidgets import widgets, Layout
from IPython.display import clear_output

import time

In [2]:
from halo import HaloNotebook as Halo

def waiting_animation(method):
    def spinner_animation(*args, **kw):
    
        spinner = Halo(text=f'Waiting for transaction...', text_color= 'cyan', color='green', spinner='dots')

        try:
            spinner.start()
            result = method(*args, **kw)
            spinner.stop_and_persist(text='Done!')
        except (KeyboardInterrupt, SystemExit):
            spinner.stop()
        
        return result    
    
    return spinner_animation

In [3]:
phrase = ''

text = widgets.Text(
    description='Phrase:',
    layout=Layout(width="70%")
)
display(text)

def handle(sender):
    global phrase
    phrase = text.value
    text.value =""
    
text.on_submit(handle)

Text(value='', description='Phrase:', layout=Layout(width='70%'))

## Derivation

In [4]:
master_key = HDPrivateKey.master_key_from_mnemonic(phrase)

del phrase

root_keys = HDKey.from_path(master_key, "m/44'/60'/0'")
acct_priv_key = root_keys[-1]
private_keys = []
addresses = []

# derivation example for Ethereum generate first 10 addresses
for i in range(10):
    keys = HDKey.from_path(acct_priv_key, '{change}/{index}'.format(change=0, index=i))
    private_key = keys[-1]
    public_key = private_key.public_key
    private_keys.append(private_key)
    addresses.append(private_key.public_key.address())
    

df = pd.DataFrame(addresses, columns = ['Address']) 

df

,Address
0,0x88a007ec4f1819f24c0988fc9c26496b99b436d1
1,0xab33d517b6a63a0b1c099b8438d6641cf1a984cc
2,0xb76d3a5d568f339411a6f0697694cddb6396df58
3,0xcb64cd644d24b57a96287f61dffeb402b01e23c6
4,0x8396738b4cb2a2a7ca636161346a7cb22ee06e25
5,0xbd666c0ec86628475b1602aba0ebf45570a33d4d
6,0x46ff31e07d54a28c468c300325afcdac4c6b7f2e
7,0x44874a7d255df37f5b9c91d173bee6ab4441c881
8,0x9a2c78cad9e6c0dc46820c37704065bb22e5eab0
9,0x577c31aad54366e7f8cf931a804f409d20b6928c


## Ethereum transction
### Setup Web3

In [5]:
# use infura endpoint
# endpoint_url = 'wss://ropsten.infura.io/ws/adc06f70568e46d88376a8a3a30e5497'
endpoint_url = 'https://ropsten.infura.io/v3/adc06f70568e46d88376a8a3a30e5497'

# Ropsten chainId is 3
chainId = 3

# setup connection
w3 = Web3(Web3.HTTPProvider(endpoint_url))
if not w3.isConnected():
    print('Can\'t connect to Web3 endpoint')

# select gas price strategy
w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

### Get last block

In [6]:
block = w3.eth.getBlock('latest')
print(f'Current block: {block}')

Current block: AttributeDict({'difficulty': 748035821, 'extraData': HexBytes('0xde8302050d8f5061726974792d457468657265756d86312e33382e30826c69'), 'gasLimit': 8000029, 'gasUsed': 3469584, 'hash': HexBytes('0xbfdb4322b36888dbd45e77c0dce2271f6bbd5fb104e207ff0f52161879e7318f'), 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000040000000000080000000000000000000000000000000000000000040000000000000000000000000100000000000000000000040000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000008000000000000000000000000000000000000000000000000000100000000000800000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000020000000000000000000000000000000'), 'miner': '0x635B4764D1939DfAcD3a8014726159abC277BecC', 'mixHash': HexBytes('0xeb269060791a6997df28b08cc85744cbf712ebb6cf7ac9a699fa53201985ccad'), 'nonce': HexBytes('0x1edd3ea373144d98'

### Prepare accounts

In [7]:
# take first two accounts as from and to
from_address = w3.toChecksumAddress(private_keys[0].public_key.address())
from_private_key = private_keys[0]._key.to_hex()

to_address = w3.toChecksumAddress(private_keys[1].public_key.address())
to_private_key = private_keys[1]._key.to_hex()

In [8]:
@waiting_animation
def wait_for_transaction(transaction_hash):
    while True:
        transaction = w3.eth.getTransaction(transaction_hash)
        if transaction['blockNumber']:
            return transaction
        time.sleep(5)

### Send Ethereum

In [9]:
# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

transaction = {
    'to': to_address,
    'value': 200_000, # 0.0000000000002 Ether
    'gas': 21_000,
    'gasPrice': w3.eth.generateGasPrice(),
    'nonce': nonce,
    'chainId': chainId
}

signed = w3.eth.account.sign_transaction(transaction, from_private_key)

print(f'Signed transaction: {hexlify(signed.rawTransaction)}')

transaction_hash = w3.eth.sendRawTransaction(signed.rawTransaction)

print(f'Transaction hash: {hexlify(transaction_hash)}')

print(f'Ropsten link: https://ropsten.etherscan.io/tx/0x' + str(hexlify(transaction_hash), "utf-8"))

Signed transaction: b'f8688204058451bea5ef82520894ab33d517b6a63a0b1c099b8438d6641cf1a984cc83030d40802aa0b9fa530888952ed576c8dc28874e580ca6ca5a8fe72cb9424c12b279a326bbeba026b2c50387a3c38af29055525bc33b14c586b9ddaf073a5af56dda98f3fddf69'
Transaction hash: b'cea4bcf2a4056317c11f5dc1f8755787bc0895c8fe1f933da615f0d1d691b0af'
Ropsten link: https://ropsten.etherscan.io/tx/0xcea4bcf2a4056317c11f5dc1f8755787bc0895c8fe1f933da615f0d1d691b0af


In [10]:
print(wait_for_transaction(transaction_hash))

Output()

AttributeDict({'blockHash': HexBytes('0x1989e93d1f13176a2d8682ad8e80ac3ee99a3ee5984d6e56f96e5984b109d237'), 'blockNumber': 7162711, 'from': '0x88a007eC4F1819F24C0988fc9C26496b99b436D1', 'gas': 21000, 'gasPrice': 1371448815, 'hash': HexBytes('0xcea4bcf2a4056317c11f5dc1f8755787bc0895c8fe1f933da615f0d1d691b0af'), 'input': '0x', 'nonce': 1029, 'r': HexBytes('0xb9fa530888952ed576c8dc28874e580ca6ca5a8fe72cb9424c12b279a326bbeb'), 's': HexBytes('0x26b2c50387a3c38af29055525bc33b14c586b9ddaf073a5af56dda98f3fddf69'), 'to': '0xab33D517b6A63A0B1C099b8438D6641cF1a984cC', 'transactionIndex': 43, 'v': 42, 'value': 200000})


### Ethereum balance

In [11]:
addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
balances = [w3.fromWei(w3.eth.getBalance(address), 'ether') for address in addresses]

df = pd.DataFrame(zip(addresses,balances), columns = ['Address', 'Balance']) 

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,2.507656510927955213
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,0.955792922714265816
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,2.154467975998
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,0.998700340002
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,3.979769
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,3.01
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,4
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,4.1
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,3
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,2.99948678


### Send ERC20

In [12]:
# read abi for ERC20 contract
with open('erc20_abi.json', 'r') as file:
    erc20_abi = file.read()

In [13]:
# Another Test Coin (ATC) ERC20
erc20_contract_address = "0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b"
erc20_contract = w3.eth.contract(address=erc20_contract_address, abi=erc20_abi)

# fetching nonce (count of transactions)
nonce = w3.eth.getTransactionCount(from_address)

erc20_transaction = erc20_contract.functions.transfer(erc20_contract_address, 1).buildTransaction(
    {
        'chainId': chainId,
        'gas': 100_000,
        'gasPrice': w3.eth.generateGasPrice(),
        'nonce': nonce,
    })

print(erc20_transaction)

signed_erc20_transaction = w3.eth.account.sign_transaction(erc20_transaction, from_private_key)

print(f'Signed transaction: {hexlify(signed_erc20_transaction.rawTransaction)}')

erc20_transaction_hash = w3.eth.sendRawTransaction(signed_erc20_transaction.rawTransaction)

print(f'Transaction hash: {hexlify(erc20_transaction_hash)}')

print(f'Ropsten link: https://ropsten.etherscan.io/tx/0x' + str(hexlify(erc20_transaction_hash), "utf-8"))

{'value': 0, 'chainId': 3, 'gas': 100000, 'gasPrice': 1363580310, 'nonce': 1030, 'to': '0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b', 'data': '0xa9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b0000000000000000000000000000000000000000000000000000000000000001'}
Signed transaction: b'f8ab8204068451469596830186a09496f55b7d683eb46ff23abef2afb41eb54fcf524b80b844a9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b00000000000000000000000000000000000000000000000000000000000000012aa0dbc1fb61b517203d226e626a0eb1a1d6b6fdcfe1062b82f63466826b5ab4f259a05fc173645228489f81d895108370d087de04dcb6ff72b81ea8ead3c485737275'
Transaction hash: b'11f3793dad9d5241359aa29bdecf2b104307cb6ec8861d3201ac50986e2a4eb6'
Ropsten link: https://ropsten.etherscan.io/tx/0x11f3793dad9d5241359aa29bdecf2b104307cb6ec8861d3201ac50986e2a4eb6


In [14]:
print(wait_for_transaction(erc20_transaction_hash))

Output()

AttributeDict({'blockHash': HexBytes('0x0b7e8662d07c17881526e79d801ac02589de8b2f8e4efcf6f5001f77bebe62a6'), 'blockNumber': 7162716, 'from': '0x88a007eC4F1819F24C0988fc9C26496b99b436D1', 'gas': 100000, 'gasPrice': 1363580310, 'hash': HexBytes('0x11f3793dad9d5241359aa29bdecf2b104307cb6ec8861d3201ac50986e2a4eb6'), 'input': '0xa9059cbb00000000000000000000000096f55b7d683eb46ff23abef2afb41eb54fcf524b0000000000000000000000000000000000000000000000000000000000000001', 'nonce': 1030, 'r': HexBytes('0xdbc1fb61b517203d226e626a0eb1a1d6b6fdcfe1062b82f63466826b5ab4f259'), 's': HexBytes('0x5fc173645228489f81d895108370d087de04dcb6ff72b81ea8ead3c485737275'), 'to': '0x96f55B7d683EB46FF23aBEf2aFb41EB54FCF524b', 'transactionIndex': 1, 'v': 42, 'value': 0})


### ERC20 balances

In [15]:
decimals = 10 ** erc20_contract.functions.decimals().call()

erc20_addresses = [w3.toChecksumAddress(key.public_key.address()) for key in private_keys]
erc20_balances = [(erc20_contract.functions.balanceOf(address).call() / decimals) for address in erc20_addresses]

df = pd.DataFrame(zip(erc20_addresses, erc20_balances), columns=['Address', 'Balance'])

df

,Address,Balance
0,0x88a007eC4F1819F24C0988fc9C26496b99b436D1,6681.173300
1,0xab33D517b6A63A0B1C099b8438D6641cF1a984cC,179.315696
2,0xb76d3a5D568f339411a6f0697694cDDb6396df58,217.092688
3,0xcb64cD644D24b57a96287F61DfFeb402B01E23c6,112.454000
4,0x8396738b4Cb2A2a7Ca636161346a7cB22Ee06E25,109.750000
5,0xBd666C0Ec86628475b1602aBA0eBF45570A33d4d,52.740000
6,0x46FF31E07D54a28c468c300325AfcdAc4c6b7F2e,131.410000
7,0x44874A7d255dF37f5b9C91d173bEE6ab4441c881,105.320000
8,0x9a2C78cad9e6c0Dc46820C37704065bb22E5Eab0,108.510000
9,0x577C31aAd54366E7F8Cf931A804F409d20B6928C,33.590000
